In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle

In [2]:
model_type = 'chi'
buffer = pickle.load(open(f'tenhou_env/buffer/0/{model_type}.pkl', 'rb'))
buf = buffer['buffer']

In [3]:
a, b, c, d = buf[0]
print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)

(63, 34, 1)
()
(2,)
(2,)


In [6]:
b

-0.032500032

In [8]:
actor = keras.models.load_model('models/chi')

In [15]:

from keras import backend as K


ENTROPY_LOSS = 5e-3
LOSS_CLIPPING = 0.2

def proximal_policy_optimization_loss(advantage, old_prediction):
    def loss(y_true, y_pred):
        prob = K.sum(y_true * y_pred, axis=-1)
        old_prob = K.sum(y_true * old_prediction, axis=-1)
        r = prob / (old_prob + 1e-10)
        return -K.mean(K.minimum(r * advantage, K.clip(r, min_value=1-LOSS_CLIPPING, max_value=1+LOSS_CLIPPING) * advantage) + ENTROPY_LOSS * -(prob * K.log(prob + 1e-10)))
    return loss

In [16]:
from tensorflow.keras.optimizers import Adam
LR = 1e-4

state_input = keras.Input(actor.input.shape[1:])
advantage = keras.Input(shape=(1,))
old_prediction = keras.Input(shape=actor.output.shape[1:])



output = actor(state_input)
model = keras.Model(inputs=[state_input, advantage, old_prediction], outputs=[output])
model.compile(optimizer=Adam(learning_rate=LR),
                   loss=[proximal_policy_optimization_loss(
                       advantage=advantage,
                       old_prediction=old_prediction
                   )],
                   metrics=[tf.keras.metrics.Accuracy(name="accuracy")])
state_input = keras.Input(actor.input.shape[1:])
advantage = keras.Input(shape=(1,))
old_prediction = keras.Input(shape=actor.output.shape[1:])



output = actor(state_input)
model = keras.Model(inputs=[state_input, advantage, old_prediction], outputs=[output])
model.compile(optimizer=Adam(learning_rate=LR),
                   loss=[proximal_policy_optimization_loss(
                       advantage=advantage,
                       old_prediction=old_prediction
                   )],
                   metrics=[tf.keras.metrics.Accuracy(name="accuracy")])

In [18]:
idxs = np.random.randint(0, len(buf), size=128)
batch = np.asarray(buf)[idxs]
feature, advantage, old_prediction, action = zip(*batch)

feature = np.asarray(feature)
advantage = np.asarray(advantage)
old_prediction = np.asarray(old_prediction)
action = np.asarray(action)


ValueError: not enough values to unpack (expected 4, got 0)

In [ ]:
print(f"batch shape: feature:{feature.shape}, advantage:{advantage.shape}")
actor_loss = self.model.fit(x=[np.asarray(feature), np.asarray(advantage), np.asarray(old_prediction)], y=[np.asarray(action)], shuffle=True, epochs=EPOCHS,
                            verbose=False)
print(f"epoch: {cnt}, actor loss: {actor_loss}")